## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
import sys
sys.path.append("..")

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import csv
import os
import numpy as np

# Import API key
from config import g_key
from config import weather_api_key
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('C:\\Users\\EKANGA PATRICK\\Desktop\\Class\\World_Weather_Analysis\\Weather_Database\WeatherPy_Database.csv')

In [4]:
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tuatapere,NZ,-46.13,167.68,72.00,49,100,14.00,overcast clouds
1,1,Arraial Do Cabo,BR,-22.97,-42.03,73.40,78,40,13.87,scattered clouds
2,2,Matane,CA,48.83,-67.52,43.00,1,100,9.13,overcast clouds
3,3,Kapaa,US,22.08,-159.32,81.00,78,90,12.75,overcast clouds
4,4,Broome,US,42.25,-75.83,64.99,77,1,10.29,clear sky


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
min_temp 

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


75.0

In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Kapaa,US,22.08,-159.32,81.00,78,90,12.75,overcast clouds
5,5,Batticaloa,LK,7.71,81.69,77.63,74,0,6.67,clear sky
11,11,Vaini,TO,-21.20,-175.20,77.00,78,75,9.17,broken clouds
12,12,Airai,TL,-8.93,125.41,75.85,65,7,1.83,clear sky
16,16,Nanakuli,US,21.39,-158.15,82.40,74,1,9.17,clear sky
26,26,Hirara,JP,24.80,125.28,75.20,60,75,12.75,broken clouds
27,27,Atuona,PF,-9.80,-139.03,78.62,71,0,17.83,clear sky
28,28,Cockburn Town,TC,21.46,-71.14,83.05,79,100,16.13,overcast clouds
29,29,Bintulu,MY,3.17,113.03,77.00,94,20,2.24,few clouds
31,31,Tautira,PF,-17.73,-149.15,84.20,70,75,10.29,broken clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                209
City                   209
Country                209
Lat                    209
Lng                    209
Max Temp               209
Humidity               209
Cloudiness             209
Wind Speed             209
Current Description    209
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
clean_df = preferred_cities_df.dropna()

In [11]:
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kapaa,US,81.00,overcast clouds,22.08,-159.32,
5,Batticaloa,LK,77.63,clear sky,7.71,81.69,
11,Vaini,TO,77.00,broken clouds,-21.20,-175.20,
12,Airai,TL,75.85,clear sky,-8.93,125.41,
16,Nanakuli,US,82.40,clear sky,21.39,-158.15,
...,...,...,...,...,...,...,...
680,Mormugao,IN,75.20,mist,15.40,73.80,
686,Jawhar,SO,77.67,overcast clouds,2.78,45.50,
697,Tenenkou,ML,84.72,broken clouds,14.46,-4.92,
699,Kibala,TD,78.80,thunderstorm,9.11,18.35,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [14]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"] 
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError, AttributeError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kapaa,US,81.00,overcast clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
5,Batticaloa,LK,77.63,clear sky,7.71,81.69,Grand Star Guest House
11,Vaini,TO,77.00,broken clouds,-21.20,-175.20,Keleti Beach Resort
12,Airai,TL,75.85,clear sky,-8.93,125.41,Juvinal Martin
16,Nanakuli,US,82.40,clear sky,21.39,-158.15,Camp Pālehua
...,...,...,...,...,...,...,...
680,Mormugao,IN,75.20,mist,15.40,73.80,Coconut Creek Resort
686,Jawhar,SO,77.67,overcast clouds,2.78,45.50,cosoble spare parts
697,Tenenkou,ML,84.72,broken clouds,14.46,-4.92,
699,Kibala,TD,78.80,thunderstorm,9.11,18.35,


In [ ]:
print(type(hotels))

In [16]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=3.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")